<a href="https://colab.research.google.com/github/psui3905/DP_Logistic_regression-NN-mindset-/blob/master/Ass2_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf 
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
import pandas as pd
from keras_preprocessing.image import ImageDataGenerator
import os
import cv2
import random
import keras.backend as K

In [0]:
!git clone https://github.com/zhuqiangLu/deeplearning_dataset.git

Cloning into 'deeplearning_dataset'...
remote: Enumerating objects: 47446, done.
remote: Total 47446 (delta 0), reused 0 (delta 0), pack-reused 47446
Receiving objects: 100% (47446/47446), 1.86 GiB | 49.65 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Checking out files: 100% (47444/47444), done.


In [0]:
cd deeplearning_dataset


/content/deeplearning_dataset


In [13]:
ls

README.md                                             train2014/  train.txt
resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5  train.csv   val2014/


In [14]:
ROOT = os.getcwd()

NUM_CLASS = 20

TRAIN_LABEL_FILE = 'train.txt' 

TRAIN_LABEL_CSV = 'train.csv'

TRAIN_FILE = 'train2014'

TRAIN_DATA_DIR = os.path.join(ROOT, TRAIN_FILE)

PRETRAIN_WEIGHT = './resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

POOLING = 'max'

ACTIVATION = 'sigmoid'


METRICS = ['accuracy']

TRAIN_RATIO = 0.8

DEV_RATIO = 0.1

TEST_RATIO = 0.1

BATCH_SIZE = 32

EPOCH = 10

IMAGE_SIZE = 300

CLASSES=[]
for i in range(NUM_CLASS):
  CLASSES.append(str(i))
  



print(ROOT) #make sure you are under the deeplearning_dataset to access all data



/content/deeplearning_dataset


In [0]:
import csv

txt_file = r"train.txt"
csv_file = r"train.csv"

# use 'with' if the program isn't going to immediately terminate
# so you don't leave files open
# the 'b' is necessary on Windows
# it prevents \x1a, Ctrl-z, from ending the stream prematurely
# and also stops Python converting to / from different line terminators
# On other platforms, it has no effect
in_txt = csv.reader(open(txt_file, "r"), delimiter = '\t')
out_csv = csv.writer(open(csv_file, 'a'))
first_row = ['filename', 'labels']
out_csv.writerow(first_row)
out_csv.writerows(in_txt)

In [15]:
#split the labels by comma
df = pd.read_csv(TRAIN_LABEL_CSV)
df["labels"]=df["labels"].apply(lambda x:x.split(","))

#read labels
with open(TRAIN_LABEL_FILE, 'r') as f:
    labels = f.readlines()
    
#calculate the ratios
train_size = int(len(labels) * TRAIN_RATIO)
dev_size = int(len(labels) * DEV_RATIO)
test_size = int(len(labels) * TEST_RATIO)


#set up generators
datagen = ImageDataGenerator(
    rescale=1./255.)

test_datagen = ImageDataGenerator(rescale=1./255.)

#set up train generator
train_generator= datagen.flow_from_dataframe( 
    dataframe= df[:train_size],
    directory= TRAIN_DATA_DIR,
    x_col= "filename",
    y_col= "labels",
    batch_size= BATCH_SIZE,
    seed= 42,
    shuffle= True,
    class_mode= "categorical",
    target_size= (IMAGE_SIZE,IMAGE_SIZE))

#set up dev generator
valid_generator=test_datagen.flow_from_dataframe(
    dataframe=df[train_size:(train_size + dev_size) ],
    directory= TRAIN_DATA_DIR,
    x_col= "filename",
    y_col= "labels",
    batch_size= BATCH_SIZE,
    seed= 42,
    shuffle= True,
    class_mode= "categorical",
    target_size= (IMAGE_SIZE,IMAGE_SIZE))

#set up test
test_generator=test_datagen.flow_from_dataframe(
    dataframe=df[(train_size + dev_size): ],
    directory= TRAIN_DATA_DIR,
    x_col= "filename",
    batch_size= 1,
    seed= 42,
    shuffle= False,
    class_mode= None,
    target_size= (IMAGE_SIZE,IMAGE_SIZE))




#get pretrain weights
weights = PRETRAIN_WEIGHT

Found 25540 images belonging to 20 classes.
Found 3192 images belonging to 20 classes.
Found 3009 images.


In [17]:
#define base model
#model = Sequential()

# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
model = Sequential()
base_model  = (applications.ResNet50(include_top = False,  pooling = POOLING, weights = weights))

model.add(base_model)
#freeze the pretrained model
model.add(Dense(2048, activation = 'relu'))
model.add(Dropout(rate = 0.5))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(rate = 0.5))
model.add(Dense(20, activation = 'sigmoid'))

model.layers[0].trainable = False



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Training

In [24]:
def arg(y_true,y_pred):
  return K.cast(K.equal(K.argmax(y_true, axis=-1),
                          K.argmax(y_pred, axis=-1)),
                  K.floatx())

#optimizer
opt = optimizers.RMSprop(lr = 0.0001)
model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy', arg])

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 2048)              4196352   
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                10260     
Total params: 28,843,412
Trainable params: 5,255,700
Non-trainable params: 23,587,712
________________________________________________________

In [20]:
print(EPOCH)

10


In [39]:

model.fit_generator(train_generator, 
                    steps_per_epoch = train_size//BATCH_SIZE,
                    validation_data = valid_generator,
                    validation_steps= dev_size//BATCH_SIZE,
                    epochs = EPOCH,
                    verbose = 1
                   )




Epoch 1/10
799/799 [==============================] - 208s 261ms/step - loss: 0.1374 - acc: 0.9521 - arg: 0.5350 - val_loss: 0.1508 - val_acc: 0.9549 - val_arg: 0.5811
Epoch 2/10
799/799 [==============================] - 207s 260ms/step - loss: 0.1369 - acc: 0.9524 - arg: 0.5347 - val_loss: 0.1499 - val_acc: 0.9542 - val_arg: 0.5899
Epoch 3/10
799/799 [==============================] - 208s 260ms/step - loss: 0.1357 - acc: 0.9527 - arg: 0.5431 - val_loss: 0.1530 - val_acc: 0.9557 - val_arg: 0.5990
Epoch 4/10
799/799 [==============================] - 208s 260ms/step - loss: 0.1351 - acc: 0.9529 - arg: 0.5477 - val_loss: 0.1503 - val_acc: 0.9552 - val_arg: 0.5811
Epoch 5/10
799/799 [==============================] - 207s 260ms/step - loss: 0.1346 - acc: 0.9533 - arg: 0.5499 - val_loss: 0.1483 - val_acc: 0.9548 - val_arg: 0.5802
Epoch 6/10
799/799 [==============================] - 208s 260ms/step - loss: 0.1347 - acc: 0.9539 - arg: 0.5497 - val_loss: 0.1406 - val_acc: 0.9552 - val_arg:

In [0]:
from keras.models import load_model
model.save('my_model.h5')



In [41]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=test_generator.n//test_generator.batch_size,
verbose=1)

3009/3009 [==============================] - 56s 19ms/step


In [0]:
pred = np.array(pred)
am = np.argmax(pred, axis = -1)


In [43]:
print(pred)

[[1.14032626e-03 8.31484795e-06 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 2.08199024e-04]
 [2.57434845e-02 7.41985440e-03 6.13409281e-03 ... 2.31352448e-03
  6.35403395e-03 1.41161293e-01]
 [4.57358062e-02 9.76929069e-03 2.53221095e-01 ... 2.82343268e-01
  2.97113538e-01 4.80715871e-01]
 ...
 [1.00134194e-01 1.69914097e-01 4.70742583e-03 ... 8.47044587e-03
  2.57810652e-02 1.37428969e-01]
 [3.04579735e-05 2.98023224e-08 5.03957272e-05 ... 5.47147393e-01
  4.84250486e-01 1.30443186e-01]
 [7.84923732e-02 2.81339884e-02 3.32520306e-02 ... 7.73268938e-02
  1.11506283e-01 3.86135161e-01]]


In [0]:


pred_bool = (pred > 0.5)
predictions=[]
labels = train_generator.class_indices

labels = dict((v,k) for k,v in labels.items())
for row in pred_bool:
    l=[]
    for index,cls in enumerate(row):
        if cls:
            l.append(labels[index])
    predictions.append(",".join(l))
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index= False)

In [0]:
predictions=[]
labels = train_generator.class_indices

labels = dict((v,k) for k,v in labels.items())
for i in am:
  predictions.append(labels[i])
  
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results_2.csv",index= False)

In [46]:
test_labels = df['labels'][(train_size + dev_size):].values

t = 0
for i in range(len(predictions)):
  if predictions[i] in test_labels[i]:
    t+=1
print(t/len(predictions));

0.6706547025589897
